In [10]:
import yfinance as yf
import time
import numpy as np
import matplotlib.pyplot as plt
import matplotlib
import pandas as pd

matplotlib.use('Agg')  # Use the Agg backend

# Define the ticker symbol
ticker = "ETH-USD"

def calculate_rsi(data):
    # Calculate RSI for each point in the data
    close_prices = data['Close']
    deltas = close_prices.diff()
    ups = deltas.clip(lower=0)
    downs = -1 * deltas.clip(upper=0)
    rs = ups.rolling(window=14).mean() / downs.rolling(window=14).mean()
    rsi = 100 - (100 / (1 + rs))
    return rsi

def pointpos(x):
    if x['pivot']==1:
        return x['Low']-1e-3
    elif x['pivot']==2:
        return x['High']+1e-3
    else:
        return np.nan

def RSIpointpos(x):
    if x['RSIpivot']==1:
        return x['RSI']-1
    elif x['RSIpivot']==2:
        return x['RSI']+1
    else:
        return np.nan
def pivotid(df1, l, n1):
    if l-n1 < 0:
        return 0
    
    pividlow = 1
    pividhigh = 1
    
    for i in range(l-n1, l):
        if df1.Low[l] > df1.Low[i]:
            pividlow = 0
        if df1.High[l] < df1.High[i]:
            pividhigh = 0
    
    if pividlow:
        return 1
    elif pividhigh:
        return 2
    else:
        return 0


def RSIpivotid(df1, l, n1):
    if l-n1 < 0:
        return 0
    
    pividlow = 1
    pividhigh = 1
    
    for i in range(l-n1, l):
        if df1.RSI[l] > df1.RSI[i]:
            pividlow = 0
        if df1.RSI[l] < df1.RSI[i]:
            pividhigh = 0
    

    if pividlow:
        return 1
    elif pividhigh:
        return 2
    else:
        return 0

def divsignal(row, nbackcandles):
    backcandles = nbackcandles
    candleid = int(row.name)

    maxim = np.array([])
    minim = np.array([])
    xxmin = np.array([])
    xxmax = np.array([])

    maximRSI = np.array([])
    minimRSI = np.array([])
    xxminRSI = np.array([])
    xxmaxRSI = np.array([])

    for i in range(candleid - backcandles, candleid + 1):
        if df.iloc[i].pivot == 1:
            minim = np.append(minim, df.iloc[i].Low)
            xxmin = np.append(xxmin, i)  # could be i instead of df.iloc[i].name
        if df.iloc[i].pivot == 2:
            maxim = np.append(maxim, df.iloc[i].High)
            xxmax = np.append(xxmax, i)  # df.iloc[i].name
        if df.iloc[i].RSIpivot == 1:
            minimRSI = np.append(minimRSI, df.iloc[i].RSI)
            xxminRSI = np.append(xxminRSI, df.iloc[i].name)
        if df.iloc[i].RSIpivot == 2:
            maximRSI = np.append(maximRSI, df.iloc[i].RSI)
            xxmaxRSI = np.append(xxmaxRSI, df.iloc[i].name)

    if maxim.size < 2 or minim.size < 2 or maximRSI.size < 2 or minimRSI.size < 2:
        return 0

    slmin, intercmin = np.polyfit(xxmin, minim, 1)
    slmax, intercmax = np.polyfit(xxmax, maxim, 1)
    slminRSI, intercminRSI = np.polyfit(xxminRSI, minimRSI, 1)
    slmaxRSI, intercmaxRSI = np.polyfit(xxmaxRSI, maximRSI, 1)

    if slmin > 1e-4 and slmax > 1e-4 and slmaxRSI < -0.1:
        return 1
    elif slmin < -1e-4 and slmax < -1e-4 and slminRSI > 0.1:
        return 2
    else:
        return 0
    
    
    
# Create empty lists to store the price and RSI data
# Create empty lists to store the price and RSI data
price_data = []
rsi_data = []
n1=5
# Create a DataFrame to store the data
df = pd.DataFrame(columns=['Timestamp', 'Open', 'High', 'Low', 'Close', 'RSI', 'pivot', 'RSIpivot'])

# Create a figure and axes for the plot
fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(10, 8))

while True:
    # Fetch the latest data
    data = yf.download(tickers=ticker, period="1d", interval="1m")
    
    # Process the data as needed
    # ...
    
    # Calculate the RSI
    rsi = calculate_rsi(data)
    
    # Get the latest close price and RSI
    latest_close_price = data['Close'][-1]
    latest_open_price = data['Open'][-1]
    latest_high_price = data['High'][-1]
    latest_low_price = data['Low'][-1]
    
    latest_rsi = rsi.values[-1] if not rsi.empty else None
    
    # Append the latest price and RSI to the data lists
    price_data.append(latest_close_price)
    rsi_data.append(latest_rsi)
    
    # Add the data to the DataFrame
    timestamp = pd.Timestamp.now()
    new_data = {'Timestamp': timestamp, 'Open': latest_open_price, 'High': latest_high_price, 'Low': latest_low_price,
                'Close': latest_close_price, 'RSI': latest_rsi, 'pivot': np.nan, 'RSIpivot': np.nan}
    df = pd.concat([df, pd.DataFrame([new_data])], ignore_index=True)
    
    # Implement the RSI strategy and get the buy-sell signals
    
    
    # Update the DataFrame with the buy-sell signals
    
    # Calculate pivot points for prices and RSI
    # ... (previous code)

# Update the DataFrame with the buy-sell signals

# Calculate pivot points for prices and RSI
    pivot = pivotid(df, len(df) - 1, n1)
    RSIpivot = RSIpivotid(df, len(df) - 1, n1)

# Update the 'pivot' and 'RSIpivot' columns in the DataFrame
    df.loc[len(df) - 1, 'pivot'] = pivot
    df.loc[len(df) - 1, 'RSIpivot'] = RSIpivot

# Calculate pointpos and RSIpointpos
    df['pointpos'] = df.apply(pointpos, axis=1)
    df['RSIpointpos'] = df.apply(RSIpointpos, axis=1)

# Fit lines to pivot points for prices and RSI
    # ... (previous code)

# Fit lines to pivot points for prices and RSI
    backcandles = 60
    candleid = len(df) - 1

    maxim = np.array([])
    minim = np.array([])
    xxmin = np.array([])
    xxmax = np.array([])

    maximRSI = np.array([])
    minimRSI = np.array([])
    xxminRSI = np.array([])
    xxmaxRSI = np.array([])

    for i in range(candleid - backcandles, candleid + 1):
        if i >= 0 and i < len(df):
            if df.iloc[i].pivot == 1:
                minim = np.append(minim, df.iloc[i].Low)
                xxmin = np.append(xxmin, i)  # could be i instead df.iloc[i].name
            if df.iloc[i].pivot == 2:
                maxim = np.append(maxim, df.iloc[i].High)
                xxmax = np.append(xxmax, i)  # df.iloc[i].name
            if df.iloc[i].RSIpivot == 1:
                minimRSI = np.append(minimRSI, df.iloc[i].RSI)
                xxminRSI = np.append(xxminRSI, df.iloc[i].name)
            if df.iloc[i].RSIpivot == 2:
                maximRSI = np.append(maximRSI, df.iloc[i].RSI)
                xxmaxRSI = np.append(xxmaxRSI, df.iloc[i].name)

    slmin, intercmin = np.polyfit(xxmin, minim, 1) if xxmin.size > 0 else (0, 0)
    slmax, intercmax = np.polyfit(xxmax, maxim, 1) if xxmax.size > 0 else (0, 0)
    slminRSI, intercminRSI = np.polyfit(xxminRSI, minimRSI, 1) if xxminRSI.size > 0 else (0, 0)
    slmaxRSI, intercmaxRSI = np.polyfit(xxmaxRSI, maximRSI, 1) if xxmaxRSI.size > 0 else (0, 0)


    print(slmin, slmax, slminRSI, slmaxRSI)

    ax1.cla()  # Clear the previous plot
    ax1.plot(price_data)
    ax1.set_title(f"{ticker} Price")

# Plot the RSI chart
    ax2.cla()  # Clear the previous plot
    ax2.plot(rsi_data)
    ax2.axhline(30, linestyle='--', color='red', alpha=0.5)
    ax2.axhline(70, linestyle='--', color='green', alpha=0.5)
    ax2.set

# Save the plot as an image
    plt.savefig("divergence.png")

[*********************100%***********************]  1 of 1 completed
0 0 0 0
[*********************100%***********************]  1 of 1 completed
0 0 0 0
[*********************100%***********************]  1 of 1 completed
0 0 0 0
[*********************100%***********************]  1 of 1 completed
0 0 0 0
[*********************100%***********************]  1 of 1 completed
0 0 0 0
[*********************100%***********************]  1 of 1 completed
186.87031249999987 0 1.6209194794614326 0


/Users/niharpatel/opt/anaconda3/envs/tf_env/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3505: RankWarning: Polyfit may be poorly conditioned
  exec(code_obj, self.user_global_ns, self.user_ns)
/Users/niharpatel/opt/anaconda3/envs/tf_env/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3505: RankWarning: Polyfit may be poorly conditioned
  exec(code_obj, self.user_global_ns, self.user_ns)


[*********************100%***********************]  1 of 1 completed
-5.726361244766917e-14 0 2.6500945562659944e-15 0
[*********************100%***********************]  1 of 1 completed
4.611607469145454e-14 0 1.77624669910956e-15 0
[*********************100%***********************]  1 of 1 completed
1.5868071899542798e-13 0 7.328122555474963e-16 0
[*********************100%***********************]  1 of 1 completed
-2.8102930298760937e-14 0 -1.5687678980832854e-15 0
[*********************100%***********************]  1 of 1 completed
-2.0173107616127596e-13 0 -4.724626602950853e-16 0
[*********************100%***********************]  1 of 1 completed
6.716437100286751e-16 0 -8.580549709889901e-16 0
[*********************100%***********************]  1 of 1 completed
-8.161332269645787e-14 0 -4.758453758937206e-16 0
[*********************100%***********************]  1 of 1 completed
4.5228074306214195e-14 0 5.473293249541827e-16 0
[*********************100%***********************] 

[*********************100%***********************]  1 of 1 completed
1.6906341607301877e-15 0 2.172072004887645e-17 0
[*********************100%***********************]  1 of 1 completed
-2.75684233028738e-15 0 -2.6619559702069066e-17 0
[*********************100%***********************]  1 of 1 completed
1.5548781131928974e-14 0 1.0403611650558642e-16 0
[*********************100%***********************]  1 of 1 completed
4.026257396862869e-15 0 5.540350926396039e-17 0
[*********************100%***********************]  1 of 1 completed
8.859580367982499e-15 0 -5.829925985392219e-19 0
[*********************100%***********************]  1 of 1 completed
-2.70697016058722e-15 0 2.3276270544092902e-17 0
[*********************100%***********************]  1 of 1 completed
-6.034670487058874e-15 0 -1.2438133526178558e-16 0
[*********************100%***********************]  1 of 1 completed
-6.996385125110453e-15 0 -1.397297181623914e-16 0
[*********************100%***********************]  

/Users/niharpatel/opt/anaconda3/envs/tf_env/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3505: RankWarning: Polyfit may be poorly conditioned
  exec(code_obj, self.user_global_ns, self.user_ns)
/Users/niharpatel/opt/anaconda3/envs/tf_env/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3505: RankWarning: Polyfit may be poorly conditioned
  exec(code_obj, self.user_global_ns, self.user_ns)


...

/Users/niharpatel/opt/anaconda3/envs/tf_env/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3505: RankWarning: Polyfit may be poorly conditioned
  exec(code_obj, self.user_global_ns, self.user_ns)
/Users/niharpatel/opt/anaconda3/envs/tf_env/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3505: RankWarning: Polyfit may be poorly conditioned
  exec(code_obj, self.user_global_ns, self.user_ns)


[*********************100%***********************]  1 of 1 completed
6.703138001222186e-15 7.132454675572514 5.525442124341943e-17 0.06186715570463484


/Users/niharpatel/opt/anaconda3/envs/tf_env/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3505: RankWarning: Polyfit may be poorly conditioned
  exec(code_obj, self.user_global_ns, self.user_ns)
/Users/niharpatel/opt/anaconda3/envs/tf_env/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3505: RankWarning: Polyfit may be poorly conditioned
  exec(code_obj, self.user_global_ns, self.user_ns)


[*********************100%***********************]  1 of 1 completed
-1.670464121981533e-15 0 7.591390550678901e-18 0
[*********************100%***********************]  1 of 1 completed
-1.4192111518067499e-15 0 7.369140572469201e-18 0
[*********************100%***********************]  1 of 1 completed
-1.8556822575033942e-16 0 3.497734809348769e-18 0
[*********************100%***********************]  1 of 1 completed
-2.991522805829317e-15 0 -1.2030222507611973e-17 0
[*********************100%***********************]  1 of 1 completed
-1.158057557633848e-16 0 -1.7581519778692778e-17 0
[*********************100%***********************]  1 of 1 completed
-2.956092398531592e-15 0 -6.47012266996242e-18 0
[*********************100%***********************]  1 of 1 completed
4.528660157319442e-15 0 3.905561720620713e-17 0
[*********************100%***********************]  1 of 1 completed
5.082535486650029e-15 0 2.003655040106065e-17 0
[*********************100%***********************]  

KeyboardInterrupt: 

In [ ]:
import yfinance as yf
import time
import numpy as np
import matplotlib.pyplot as plt
import matplotlib
import pandas as pd

matplotlib.use('Agg')  # Use the Agg backend

# Define the ticker symbol
ticker = "ETH-USD"

def calculate_rsi(data):
    # Calculate RSI for each point in the data
    close_prices = data['Close']
    deltas = close_prices.diff()
    ups = deltas.clip(lower=0)
    downs = -1 * deltas.clip(upper=0)
    rs = ups.rolling(window=14).mean() / downs.rolling(window=14).mean()
    rsi = 100 - (100 / (1 + rs))
    return rsi

def pointpos(x):
    if x['pivot'] == 1:
        return x['Low'] - 1e-3
    elif x['pivot'] == 2:
        return x['High'] + 1e-3
    else:
        return np.nan

def RSIpointpos(x):
    if x['RSIpivot'] == 1:
        return x['RSI'] - 1
    elif x['RSIpivot'] == 2:
        return x['RSI'] + 1
    else:
        return np.nan

def pivotid(df1, l, n1):
    if l - n1 < 0:
        return 0

    pividlow = 1
    pividhigh = 1

    for i in range(l - n1, l):
        if df1.Low[l] > df1.Low[i]:
            pividlow = 0
        if df1.High[l] < df1.High[i]:
            pividhigh = 0

    if pividlow:
        return 1
    elif pividhigh:
        return 2
    else:
        return 0

def RSIpivotid(df1, l, n1):
    if l - n1 < 0:
        return 0

    pividlow = 1
    pividhigh = 1

    for i in range(l - n1, l):
        if df1.RSI[l] > df1.RSI[i]:
            pividlow = 0
        if df1.RSI[l] < df1.RSI[i]:
            pividhigh = 0

    if pividlow:
        return 1
    elif pividhigh:
        return 2
    else:
        return 0

def divsignal(row, nbackcandles):
    backcandles = nbackcandles
    candleid = int(row.name)

    maxim = np.array([])
    minim = np.array([])
    xxmin = np.array([])
    xxmax = np.array([])

    maximRSI = np.array([])
    minimRSI = np.array([])
    xxminRSI = np.array([])
    xxmaxRSI = np.array([])

    for i in range(candleid - backcandles, candleid + 1):
        if i >= 0 and i < len(df):
            if df.iloc[i].pivot == 1:
                minim = np.append(minim, df.iloc[i].Low)
                xxmin = np.append(xxmin, i)  # could be i instead df.iloc[i].name
            if df.iloc[i].pivot == 2:
                maxim = np.append(maxim, df.iloc[i].High)
                xxmax = np.append(xxmax, i)  # df.iloc[i].name
            if df.iloc[i].RSIpivot == 1:
                minimRSI = np.append(minimRSI, df.iloc[i].RSI)
                xxminRSI = np.append(xxminRSI, df.iloc[i].name)
            if df.iloc[i].RSIpivot == 2:
                maximRSI = np.append(maximRSI, df.iloc[i].RSI)
                xxmaxRSI = np.append(xxmaxRSI, df.iloc[i].name)

    if maxim.size < 2 or minim.size < 2 or maximRSI.size < 2 or minimRSI.size < 2:
        return 0

    slmin, intercmin = np.polyfit(xxmin, minim, 1)
    slmax, intercmax = np.polyfit(xxmax, maxim, 1)
    slminRSI, intercminRSI = np.polyfit(xxminRSI, minimRSI, 1)
    slmaxRSI, intercmaxRSI = np.polyfit(xxmaxRSI, maximRSI, 1)

    if slmin > 1e-4 and slmax > 1e-4 and slmaxRSI < -0.1:
        return 1
    elif slmin < -1e-4 and slmax < -1e-4 and slminRSI > 0.1:
        return 2
    else:
        return 0

# Create empty lists to store the price and RSI data
price_data = []
rsi_data = []
n1 = 5

# Create a DataFrame to store the data
df = pd.DataFrame(columns=['Timestamp', 'Open', 'High', 'Low', 'Close', 'RSI', 'pivot', 'RSIpivot'])

# Create a figure and axes for the plot
fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(10, 8))

while True:
    # Fetch the latest data
    data = yf.download(tickers=ticker, period="1d", interval="1m")

    # Process the data as needed
    # ...

    # Calculate the RSI
    rsi = calculate_rsi(data)

    # Get the latest close price and RSI
    latest_close_price = data['Close'][-1]
    latest_open_price = data['Open'][-1]
    latest_high_price = data['High'][-1]
    latest_low_price = data['Low'][-1]

    latest_rsi = rsi.values[-1] if not rsi.empty else None

    # Append the latest price and RSI to the data lists
    price_data.append(latest_close_price)
    rsi_data.append(latest_rsi)

    # Add the data to the DataFrame
    timestamp = pd.Timestamp.now()
    new_data = {'Timestamp': timestamp, 'Open': latest_open_price, 'High': latest_high_price, 'Low': latest_low_price,
                'Close': latest_close_price, 'RSI': latest_rsi, 'pivot': np.nan, 'RSIpivot': np.nan}
    df = pd.concat([df, pd.DataFrame([new_data])], ignore_index=True)

    # Update the DataFrame with the buy-sell signals
    pivot = pivotid(df, len(df) - 1, n1)
    RSIpivot = RSIpivotid(df, len(df) - 1, n1)
    df.loc[len(df) - 1, 'pivot'] = pivot
    df.loc[len(df) - 1, 'RSIpivot'] = RSIpivot

    # Calculate pointpos and RSIpointpos
    df['pointpos'] = df.apply(pointpos, axis=1)
    df['RSIpointpos'] = df.apply(RSIpointpos, axis=1)

    # Fit lines to pivot points for prices and RSI
    backcandles = 60
    candleid = len(df) - 1
    maxim = np.array([])
    minim = np.array([])
    xxmin = np.array([])
    xxmax = np.array([])
    maximRSI = np.array([])
    minimRSI = np.array([])
    xxminRSI = np.array([])
    xxmaxRSI = np.array([])

    for i in range(candleid - backcandles, candleid + 1):
        if i >= 0 and i < len(df):
            if df.iloc[i].pivot == 1:
                minim = np.append(minim, df.iloc[i].Low)
                xxmin = np.append(xxmin, i)  # could be i instead of df.iloc[i].name
            if df.iloc[i].pivot == 2:
                maxim = np.append(maxim, df.iloc[i].High)
                xxmax = np.append(xxmax, i)  # df.iloc[i].name
            if df.iloc[i].RSIpivot == 1:
                minimRSI = np.append(minimRSI, df.iloc[i].RSI)
                xxminRSI = np.append(xxminRSI, df.iloc[i].name)
            if df.iloc[i].RSIpivot == 2:
                maximRSI = np.append(maximRSI, df.iloc[i].RSI)
                xxmaxRSI = np.append(xxmaxRSI, df.iloc[i].name)

    slmin, intercmin = np.polyfit(xxmin, minim, 1) if xxmin.size > 0 else (0, 0)
    slmax, intercmax = np.polyfit(xxmax, maxim, 1) if xxmax.size > 0 else (0, 0)
    slminRSI, intercminRSI = np.polyfit(xxminRSI, minimRSI, 1) if xxminRSI.size > 0 else (0, 0)
    slmaxRSI, intercmaxRSI = np.polyfit(xxmaxRSI, maximRSI, 1) if xxmaxRSI.size > 0 else (0, 0)

    print(slmin, slmax, slminRSI, slmaxRSI)

    ax1.cla()  # Clear the previous plot
    ax1.plot(price_data)
    ax1.set_title(f"{ticker} Price")

    # Plot the RSI chart
    ax2.cla()  # Clear the previous plot
    ax2.plot(rsi_data)
    ax2.axhline(30, linestyle='--', color='red', alpha=0.5)
    ax2.axhline(70, linestyle='--', color='green', alpha=0.5)
    ax2.set_title(f"{ticker} RSI")

    # Save the plot as an image
    plt.savefig("divergence.png")

    # Get the latest buy-sell signal
    signal = divsignal(df.iloc[-1], nbackcandles=10)

    # Implement the buy-sell logic based on the signal
    if signal == 1:
        # Buy signal
        print("Buy signal")
        # Place your buy order code here
        # ...

    elif signal == 2:
        # Sell signal
        print("Sell signal")
        # Place your sell order code here
        # ...
    print(df.tail(1))
    # Sleep for a specific interval before fetching the next data
    time.sleep(2)  # Sleep for 60 seconds before the next iteration


[*********************100%***********************]  1 of 1 completed
0 0 0 0
                   Timestamp         Open         High          Low   
0 2023-06-30 19:34:06.368392  1857.411011  1857.411011  1857.411011  \

         Close       RSI  pivot  RSIpivot  pointpos  RSIpointpos  
0  1857.411011  48.91776    0.0       0.0       NaN          NaN  
[*********************100%***********************]  1 of 1 completed
0 0 0 0
                   Timestamp         Open         High          Low   
1 2023-06-30 19:34:09.028595  1857.411011  1857.411011  1857.411011  \

         Close       RSI  pivot  RSIpivot  pointpos  RSIpointpos  
1  1857.411011  48.91776    0.0       0.0       NaN          NaN  
[*********************100%***********************]  1 of 1 completed
0 0 0 0
                   Timestamp         Open         High          Low   
2 2023-06-30 19:34:11.741291  1857.411011  1857.411011  1857.411011  \

         Close       RSI  pivot  RSIpivot  pointpos  RSIpointpos  
2  18

/Users/niharpatel/opt/anaconda3/envs/tf_env/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3505: RankWarning: Polyfit may be poorly conditioned
  exec(code_obj, self.user_global_ns, self.user_ns)
/Users/niharpatel/opt/anaconda3/envs/tf_env/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3505: RankWarning: Polyfit may be poorly conditioned
  exec(code_obj, self.user_global_ns, self.user_ns)


[*********************100%***********************]  1 of 1 completed
4.299702646793128e-13 0 7.316348675092187e-15 0
                   Timestamp         Open         High          Low   
6 2023-06-30 19:34:21.659383  1857.411011  1857.411011  1857.411011  \

         Close       RSI  pivot  RSIpivot     pointpos  RSIpointpos  
6  1857.411011  48.91776    1.0       1.0  1857.410011     47.91776  
[*********************100%***********************]  1 of 1 completed
4.927992811572679e-14 0 3.5989908322919443e-16 0
                   Timestamp         Open         High          Low   
7 2023-06-30 19:34:24.081363  1857.411011  1857.411011  1857.411011  \

         Close       RSI  pivot  RSIpivot     pointpos  RSIpointpos  
7  1857.411011  48.91776    1.0       1.0  1857.410011     47.91776  
[*********************100%***********************]  1 of 1 completed
1.998668413490098e-13 0 7.903823683154909e-16 0
                   Timestamp         Open         High          Low   
8 2023-06-3

[*********************100%***********************]  1 of 1 completed
-0.126295730530541 0 -0.08208799075317745 0
                    Timestamp         Open         High          Low   
27 2023-06-30 19:35:15.261752  1854.904907  1854.904907  1854.904907  \

          Close        RSI  pivot  RSIpivot     pointpos  RSIpointpos  
27  1854.904907  47.288876    1.0       1.0  1854.903907    46.288876  
[*********************100%***********************]  1 of 1 completed
-0.11767790421192562 0 -0.07648669256060692 0
                    Timestamp         Open         High          Low   
28 2023-06-30 19:35:17.723165  1854.904907  1854.904907  1854.904907  \

          Close        RSI  pivot  RSIpivot     pointpos  RSIpointpos  
28  1854.904907  47.288876    1.0       1.0  1854.903907    46.288876  
[*********************100%***********************]  1 of 1 completed
-0.10988300030052707 0 -0.07142026634398681 0
                    Timestamp         Open         High          Low   
29 2023

/Users/niharpatel/opt/anaconda3/envs/tf_env/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3505: RankWarning: Polyfit may be poorly conditioned
  exec(code_obj, self.user_global_ns, self.user_ns)
/Users/niharpatel/opt/anaconda3/envs/tf_env/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3505: RankWarning: Polyfit may be poorly conditioned
  exec(code_obj, self.user_global_ns, self.user_ns)


...

[*********************100%***********************]  1 of 1 completed
0.04856415783910018 -6.440079231155731e-15 -0.1332385672933333 0.21696331002659353
                     Timestamp         Open         High          Low   
149 2023-06-30 19:40:28.101305  1849.973145  1849.973145  1849.973145  \

           Close        RSI  pivot  RSIpivot     pointpos  RSIpointpos  
149  1849.973145  54.666777    1.0       1.0  1849.972145    53.666777  


/Users/niharpatel/opt/anaconda3/envs/tf_env/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3505: RankWarning: Polyfit may be poorly conditioned
  exec(code_obj, self.user_global_ns, self.user_ns)


[*********************100%***********************]  1 of 1 completed
0.05054191996756489 -6.440079231155731e-15 -0.12704059243365073 0.21696331002659353
                     Timestamp         Open         High          Low   
150 2023-06-30 19:40:30.582395  1849.973145  1849.973145  1849.973145  \

           Close        RSI  pivot  RSIpivot     pointpos  RSIpointpos  
150  1849.973145  54.666777    1.0       1.0  1849.972145    53.666777  


/Users/niharpatel/opt/anaconda3/envs/tf_env/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3505: RankWarning: Polyfit may be poorly conditioned
  exec(code_obj, self.user_global_ns, self.user_ns)


[*********************100%***********************]  1 of 1 completed
0.05223804253147536 0.02681964474047587 -0.12358729583018066 0.31544909761240075
                     Timestamp         Open         High          Low   
151 2023-06-30 19:40:33.105254  1850.562744  1850.562744  1850.562744  \

           Close        RSI  pivot  RSIpivot     pointpos  RSIpointpos  
151  1850.562744  62.797937    2.0       2.0  1850.563744    63.797937  
[*********************100%***********************]  1 of 1 completed
0.054018879956768386 0.026901077920121995 -0.11955465976469928 0.3083186930551586
                     Timestamp         Open         High          Low   
152 2023-06-30 19:40:35.534090  1850.562744  1850.562744  1850.562744  \

           Close        RSI  pivot  RSIpivot     pointpos  RSIpointpos  
152  1850.562744  62.797937    2.0       2.0  1850.563744    63.797937  
[*********************100%***********************]  1 of 1 completed
0.055890282317800424 0.02649733206456887 -0.